<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/%E5%A4%9A%E9%A0%85%E9%81%B8%E6%93%87/demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [2]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments,AutoModelForMultipleChoice


In [3]:
datasets = load_dataset('roberthsu2003/for_Multiple_Choice')
datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/504 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/364k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5856 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1825 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'choices', 'answer'],
        num_rows: 5856
    })
    validation: Dataset({
        features: ['context', 'question', 'choices', 'answer'],
        num_rows: 1825
    })
})

## 數據集預處理

In [4]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
#只使用一筆資料的測試
from pprint import pprint
one_dataset = datasets['train'].select(range(1)) #one_dataset資料類型是Dataset
print(one_dataset)
contexts = []
question_choice = []
labels = []
ctx = one_dataset['context'][0]
question = one_dataset['question'][0]
answer = one_dataset['answer'][0]
choices = one_dataset["choices"][0]
for choice in choices:
    contexts.append(ctx)
    question_choice.append(question + " " + choice)

if len(choices) < 4:
    for _ in range(4 - len(choices)):
        contexts.append(ctx)
        question_choice.append(question + " " + "不知道")
labels.append(choices.index(answer))


tokenizer_example = tokenizer(contexts, question_choice, truncation="only_first", max_length=256, padding="max_length")
tokenizer_example['labels'] = labels
tokenized_example = {k: [v[i:i+4] for i in range(0,len(v),4)] for k, v in tokenizer_example.items()}
pprint(tokenized_example,compact=True)

Dataset({
    features: ['context', 'question', 'choices', 'answer'],
    num_rows: 1
})
{'attention_mask': [[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0,

In [5]:
#一次1筆資料使用function的測試

def process_function(examples):
    contexts = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = examples['context'][idx]
        question = examples['question'][idx]

        choices = examples["choices"][idx]
        for choice in choices:
            contexts.append(ctx)
            question_choice.append(question + " " + choice)

        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                contexts.append(ctx)
                question_choice.append(question + " " + "不知道")
        answer = examples['answer'][idx]
        labels.append(choices.index(answer))
    tokenizer_example = tokenizer(contexts, question_choice, truncation="only_first", max_length=256, padding="max_length")
    tokenized_example = {k: [v[i:i+4] for i in range(0,len(v),4)] for k, v in tokenizer_example.items()}
    tokenizer_example['labels'] = labels
    return tokenized_example


In [6]:
#測試10筆
res = datasets["train"].select(range(10)).map(process_function, batched=True)
res

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'question', 'choices', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10
})

In [7]:
# 檢查維度
import numpy as np
np.array(res['input_ids']).shape

(10, 4, 256)

In [8]:
tokenized_c3 = datasets.map(process_function, batched=True,remove_columns=datasets['train'].column_names)
tokenized_c3

Map:   0%|          | 0/5856 [00:00<?, ? examples/s]

Map:   0%|          | 0/1825 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5856
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1825
    })
})

### 建立模型

In [9]:
model = AutoModelForMultipleChoice.from_pretrained('google-bert/bert-base-chinese')

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 建立評估函數

In [10]:
import numpy as np
accuracy = evaluate.load("accuracy")

def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions,axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

##測試評估函式compute_metric(pred)
# 假資料
# 假設我們有 5 個樣本，每個樣本有 4 個選項
num_examples = 5
num_choices = 4

# 建立假預測分數
# 每一行代表一個樣本，每一列代表一個選項的分數
dummy_predictions = np.random.rand(num_examples, num_choices)
print(dummy_predictions)

# 建立假標籤（正確答案）
# 每個元素是正確選項的索引（0, 1, 2 或 3）
dummy_labels = np.random.randint(0, num_choices, num_examples)
print(dummy_labels)
# 組合成預期的輸入格式
dummy_pred = (dummy_predictions, dummy_labels)

# 測試函數
result = compute_metric(dummy_pred)
print(result)

[[0.17632336 0.45467484 0.94328666 0.71431541]
 [0.15896865 0.13044475 0.72087614 0.46465121]
 [0.08456302 0.93748855 0.05461385 0.48524885]
 [0.8138252  0.80592429 0.31478081 0.21398716]
 [0.66049254 0.09279617 0.48874519 0.53397268]]
[1 2 2 3 3]
{'accuracy': 0.2}


## 配置訓練參數

In [11]:
args = TrainingArguments(
    output_dir="./multiple_choice",
    per_device_train_batch_size=16,#因為是3維,16*4=64,實際會是64
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    report_to='none'
)

## 建立訓練器

In [12]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset=tokenized_c3['train'],
    eval_dataset=tokenized_c3['validation'],
    compute_metrics=compute_metric
)

## 訓練模型


In [13]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
## 模型預測
from typing import Any
import torch

class MultipleChoicePipeline:
    def __init__(self, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, context, question, choices):
        cs, qcs = [], []
        for choice in choices:
            cs.append(context)
            qcs.append(question + " " + choice)
        return tokenizer(cs, qcs, truncation="only_first", max_length=256, return_tensors="pt")

    def predict(self, inputs):
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        predition = torch.argmax(logits, dim=-1).cpu().item()
        return choices[predition]

    def __call__(self, context, question, choices) -> Any:
        inputs = self.preprocess(context,question,choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [ ]:
pipe = MultipleChoicePipeline(model, tokenizer)


In [ ]:
pipe("國堂在台北上班","國堂在哪裏上班?",['台北','台中'])

AttributeError: 'Tensor' object has no attribute 'cup'